<a href="https://colab.research.google.com/github/Zilleplus/MachineLearning/blob/feature%2Fadd-HML-chap15-notebooks/HML/chapter15/WaveNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
print("using tensorflow version: " + tf.__version__)

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offset1, offset2, = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offset1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offset2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)


n_steps = 100
n_samples = 10000
series = generate_time_series(batch_size=n_samples, n_steps=n_steps + 1)
x_train, y_train = series[:7000, :n_steps], series[:7000, -1]
x_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
x_test, y_test = series[9000:, :n_steps], series[9000:, -1]

using tensorflow version: 2.7.0


In [21]:
x_train.shape

(7000, 100, 1)

In [23]:
y_train.shape

(7000, 1)

In [46]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))

for rate in (1, 2, 4, 8)*2:
  model.add(keras.layers.Conv1D(filters=20,kernel_size=2,padding="causal", activation="relu", dilation_rate=rate))

model.add(keras.layers.Conv1D(filters=1, kernel_size=1)) # filters=1 as we only predict 1 value,  in the book we do 10, so filters=10
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=last_time_step_mse)

In [47]:
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 14s 50ms/step - loss: 0.1438 - last_time_step_mse: 0.1422 - val_loss: 0.1396 - val_last_time_step_mse: 0.1398
Epoch 2/20
219/219 [==============================] - 7s 30ms/step - loss: 0.1432 - last_time_step_mse: 0.1419 - val_loss: 0.1392 - val_last_time_step_mse: 0.1389
Epoch 3/20
219/219 [==============================] - 6s 28ms/step - loss: 0.1429 - last_time_step_mse: 0.1412 - val_loss: 0.1388 - val_last_time_step_mse: 0.1364
Epoch 4/20
219/219 [==============================] - 6s 28ms/step - loss: 0.1423 - last_time_step_mse: 0.1404 - val_loss: 0.1388 - val_last_time_step_mse: 0.1354
Epoch 5/20
219/219 [==============================] - 6s 28ms/step - loss: 0.1421 - last_time_step_mse: 0.1403 - val_loss: 0.1383 - val_last_time_step_mse: 0.1378
Epoch 6/20
219/219 [==============================] - 6s 28ms/step - loss: 0.1421 - last_time_step_mse: 0.1400 - val_loss: 0.1382 - val_last_time_step_mse: 0.1351
Epoch 7/20
219/219 [=

In [28]:
print(x_valid.shape)
print(y_valid.shape)

(2000, 100, 1)
(2000, 1)
